In [1]:
import streamlit as st
import pandas as pd
import sqlalchemy as sq
import datetime as dt

In [2]:
import numpy as np

In [5]:
np.random.randint(0,10)

3

In [2]:
import re

In [4]:
re.findall(('\d\d'),'aa 2a 32')

['32']

In [2]:
'Who is here'.split(',')

['Who is here']

# Aviationstack API

In [14]:
import requests as r

In [67]:
api_access_key = '114c76f3f6cc7f559ecf5fd938a28c72'

## Airline Data

[Source](https://aviationstack.com/documentation)

In [71]:
url = 'http://api.aviationstack.com/v1/airlines'

In [72]:
params = {
    'access_key':api_access_key,
}

In [73]:
response = r.get(url = url, params = params)
response

<Response [200]>

In [95]:
list(response.json()['data'][1].keys())

['fleet_average_age',
 'callsign',
 'hub_code',
 'iata_code',
 'icao_code',
 'country_iso2',
 'date_founded',
 'iata_prefix_accounting',
 'airline_name',
 'country_name',
 'fleet_size',
 'status',
 'type']

In [92]:
for i in range(100):
    data = response.json()['data'][i]
    if 'United' in data['airline_name']:
        print(data['airline_name'],' = ',data['iata_code'])

United Airlines  =  UA


## Flight Data

In [107]:
url = 'http://api.aviationstack.com/v1/flights'

In [4]:
params = {
    'access_key':api_access_key,
    'airline_iata': 'UA',
    'flight_number':'2023',
    'dep_iata':'ORD',
    'arr_icao':'RSW'
}

NameError: name 'api_access_key' is not defined

In [118]:
response = r.get(url = url, params = params)
response

<Response [200]>

In [120]:
response.json()

{'pagination': {'limit': 100, 'offset': 0, 'count': 0, 'total': 0}, 'data': []}

# Databases

In [3]:
engine = sq.create_engine('sqlite:///data/find_me.db')
cnx = engine.connect()

In [4]:
df = pd.DataFrame({
    'current_loc':['current city'],
    'current_date':[dt.datetime.now().date()],
    'future_loc':['new city'],
    'future_date':['April 1; April 28'],
    'confirm_code':['unknown'],
    'message':['unknown']
})

In [14]:
# df.to_sql('location',con=cnx,if_exists='replace',index=False)

In [13]:
pd.read_sql('location',cnx)

,current_loc,current_date,future_loc,future_date,confirm_code,message
0,current city,2021-03-07,new city,April 1; April 28,unknown,unknown


In [7]:
new_df = pd.DataFrame({
    'plane_code':['unknown'],
    'flight_num':['unknown'],
    'year_depart':['unknown'],
    'month_depart':['unknown'],
    'day_depart':['unknown'],
    'confirm_code':['unknown'],
    'date_depart':[dt.datetime.strptime('02-27-2021 23:59','%m-%d-%Y %H:%M')],
    'date_added':[dt.datetime.now()]
})

In [15]:
# new_df.to_sql('ticket_info',con=cnx,if_exists='replace',index=False)

In [9]:
df = pd.read_sql('ticket_info', con=cnx)
df

,plane_code,flight_num,year_depart,month_depart,day_depart,confirm_code,date_depart,date_added
0,unknown,unknown,unknown,unknown,unknown,unknown,2021-02-27 23:59:00,2021-03-07 15:06:19.741900


In [10]:
df[df['date_depart'] >= dt.datetime.now()]

,plane_code,flight_num,year_depart,month_depart,day_depart,confirm_code,date_depart,date_added


In [11]:
pd.read_sql('airline_info', con=cnx)

,Airline,Country,Call Sign,IATA,ICAO
0,Alaska Airlines,USA,None,AS,ASA
1,American Airlines,USA,None,AA,AAL
2,Air Canada,Canada,None,AC,ACA
3,Aeromexico,Mexico,None,AM,AMX
4,Continental Airlines,USA,None,CO,COA
...,...,...,...,...,...
198,Viva Air Colombia,None,VIVACOLOMBIA,None,None
199,VivaAerobús,None,AEROENLACES,None,None
200,WestJet,None,WESTJET,None,None
201,Wizz Air,None,WIZZAIR,None,None


# OCR

In [14]:
import pandas as pd
import streamlit as st
import re
import datetime as dt

def airline_abbrev_scrape():
    # Read in airline abbreviations
    cols = ['Airline', 'Country', 'IATA/ICAO Codes']
    df = pd.DataFrame(columns = cols)
    airlines = {}

    i = 0
    for x in pd.read_html('https://abbreviations.yourdictionary.com/articles/airline-abbreviations-for-major-carriers.html',header=0):
        i += 1
        try:
            x.columns = cols # some columns had spaces in them
        except:
            pass
        airlines[i] = x

    for i in [1,2,3,4,5,6,7]:
        df = df.append(airlines[i])

    # Split code types
    new = df['IATA/ICAO Codes'].str.split('/', expand = True)
    new.columns = ['IATA', 'ICAO']
    df = pd.concat([df, new], axis=1).drop('IATA/ICAO Codes',axis=1)

    # Add Frontier Airlines, which was missing from tables
    df = df.append({'Airline':'Frontier Airlines','Country':'USA','IATA':'F9','ICAO':'FFT'}, ignore_index=True)

    # Remove spaces
    df['IATA'] = df['IATA'].str.strip()
    return df


In [16]:
df = airline_abbrev_scrape()

In [18]:
df.to_sql('airline_info',con=cnx,index=False,if_exists='replace')

# TripIt Integration

[Source](https://github.com/tripit/api/issues/240) for getting oauth key

In [49]:
api_id = "8fd219b1-32b8-42b6-976a-9f52b10dd1d8"
api_key = "38edb5712ffea44d695450e6bbb045a5be0ee0b4"
api_secret = "ebcfec1bde291713abff9258b63c3eba3b7a94de"

oauth_token = "8e6c121df927492ba893b1d48d333bca"
oauth_token_secret="a0836bb8c12143609fb6d2e4253eeed3"

In [46]:
import requests as r
from requests_oauthlib import OAuth1Session

`curl -k -D /dev/tty --user <username>:<password> https://api.tripit.com/v1/get/note/id/586853`

In [48]:
OAuth1Session(client_key = api_key, client_secret = api_secret).get(url)

<Response [401]>

In [ ]:
# request token

url = 'https://api.tripit.com/oauth/request_token'
params = {
    'oauth_consumer_key':
}
r.post(url=url)

In [8]:
obj_type = 'air'

In [17]:
test = r.get(url = f"https://www.tripit.com/oauth/authorize?oauth_token={oauth_token}&oauth_callback=https://www.tripit.com/trips")

In [44]:
headers_dict = {
    'oauth_consumer_key':api_key,
    'oauth_token':oauth_token,
    'oauth_signature_method':"HMAC-SHA1",
    'oauth_signature':"request signature",
    'oauth_timestamp':"1229622199",
    'oauth_nonce':"9b308304f41a4ac04fb4c4ce980c5277",
    'oauth_version':"1.0"
}

In [45]:
url = "https://api.tripit.com/v1/list/object"
r.get(url=url,headers=headers_dict)

<Response [401]>

In [39]:
from requests_oauthlib import OAuth1Session

In [43]:
OAuth1Session(api_key, api_secret).get(url)

<Response [401]>